In [1]:
import numpy as np
import partridge as ptg
import pandas as pd

In [2]:
path = 'data/SFMTA-2017-06-29.zip'

In [3]:
service_ids_by_date = ptg.read_service_ids_by_date(path)
trip_counts_by_date = ptg.read_trip_counts_by_date(path)

busiest_date, trip_count = max(trip_counts_by_date.items(), key=lambda p: p[1])
busiest_date, trip_count, list(service_ids_by_date[busiest_date])

(datetime.date(2017, 6, 12), 11856, ['1'])

In [4]:
feed = ptg.feed(path, view={
    'trips.txt': {
        'service_id': service_ids_by_date[busiest_date],
    },
})

In [5]:
feed.routes[feed.routes.route_short_name=='33']

,route_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color
39,12097,SFMTA,33,ASHBURY-18TH ST,NaN,3,NaN,NaN,NaN


In [6]:
trips = feed.trips[feed.trips.route_id=='12097'] # 33 Ashbury-18th St
trips.head()

,route_id,service_id,trip_id,trip_headsign,direction_id,block_id,shape_id
13766,12097,1,7505512,Potrero + 25th Street,0,3308,145375
13767,12097,1,7505511,Potrero + 25th Street,0,3307,145375
13768,12097,1,7505510,Potrero + 25th Street,0,3305,145375
13769,12097,1,7505509,Potrero + 25th Street,0,3304,145375
13791,12097,1,7505508,Potrero + 25th Street,0,3305,145376


In [7]:
stimes = feed.stop_times[feed.stop_times.trip_id.isin(trips.trip_id)]

stimes = stimes[
    (stimes.arrival_time >= 15*60*60)  # 3pm
    & (stimes.arrival_time <= 18*60*60) # 6pm
]

print(set(stimes.trip_id))
stimes.head()

{'7505462', '7505459', '7505463', '7505544', '7505464', '7505470', '7505474', '7505547', '7505467', '7505549', '7505471', '7505472', '7505466', '7505545', '7505551', '7505538', '7505541', '7505543', '7505465', '7505548', '7505460', '7505540', '7505542', '7505537', '7505468', '7505539', '7505473', '7505546', '7505461', '7505469', '7505550'}


,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled
492292,7505459,64500.0,64500.0,6293,1,NaN,NaN,NaN,NaN
492293,7505459,64563.0,64563.0,3879,2,NaN,NaN,NaN,NaN
492294,7505459,64604.0,64604.0,3852,3,NaN,NaN,NaN,NaN
492295,7505459,64667.0,64667.0,3644,4,NaN,NaN,NaN,NaN
492296,7505459,64747.0,64747.0,3645,5,NaN,NaN,NaN,NaN


In [8]:
tstimes = pd.merge(trips, stimes, how='inner', on='trip_id')
tstimes = pd.merge(tstimes, feed.stops, how='inner', on='stop_id')

tstimes = tstimes.sort_values(['stop_sequence', 'arrival_time', 'departure_time'])

tstimes.head()[['route_id', 'trip_id', 'trip_headsign', 'direction_id',
                'stop_sequence', 'arrival_time']]

,route_id,trip_id,trip_headsign,direction_id,stop_sequence,arrival_time
541,12097,7505548,Sacramento + Cherry,1,1,54120.0
6,12097,7505470,Potrero + 25th Street,0,1,54600.0
542,12097,7505547,Sacramento + Cherry,1,1,55020.0
7,12097,7505469,Potrero + 25th Street,0,1,55500.0
543,12097,7505546,Sacramento + Cherry,1,1,55920.0


In [9]:
inbound = tstimes[
    (tstimes.direction_id==0)
    & (tstimes.stop_name=='ASHBURY ST & CLIFFORD TER')
]

In [10]:
inbound.head()[['trip_headsign', 'stop_name', 'arrival_time']]

,trip_headsign,stop_name,arrival_time
197,Potrero + 25th Street,ASHBURY ST & CLIFFORD TER,54791.0
198,Potrero + 25th Street,ASHBURY ST & CLIFFORD TER,55691.0
199,Potrero + 25th Street,ASHBURY ST & CLIFFORD TER,56591.0
200,Potrero + 25th Street,ASHBURY ST & CLIFFORD TER,57491.0
201,Potrero + 25th Street,ASHBURY ST & CLIFFORD TER,58391.0


In [11]:
wait_seconds = inbound.arrival_time[1:].values - inbound.arrival_time[:-1].values
wait_minutes = wait_seconds / 60
wait_minutes

array([ 15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.,  15.])